In [3]:
import glob
import numpy as np
import re

# 1) 파일 목록을 숫자 기준으로 정렬하기 위한 함수
def extract_layer(fpath):
    # 'imp' 다음에 나오는 숫자를 추출
    m = re.search(r'imp(\d+)_gpu_label', fpath)
    return int(m.group(1)) if m else float('inf')

# 2) glob 으로 뽑아온 뒤 숫자 기준으로 sort
file_pattern = "../assets/xyz/imp*_gpu_label*.xyzc"
file_list = glob.glob(file_pattern)
file_list.sort(key=extract_layer)

all_data = []
for idx, fp in enumerate(file_list, start=1):
    data = np.loadtxt(fp)          # (N,4) 배열
    data[:, 3] = float(idx)        # 4번째 컬럼을 1.0, 2.0, … 로 덮어쓰기
    all_data.append(data)

if all_data:
    merged = np.vstack(all_data)
    output_path = "../assets/xyz/merged_imp_gpu_label.xyzc"
    # 좌표는 e-표기, 레이어 값만 소수점 다섯 자리로 출력
    fmt = ['%.18e', '%.18e', '%.18e', '%.5f']
    np.savetxt(output_path, merged, fmt=fmt)
    print(f"모든 파일을 합쳐서 저장 완료: {output_path}")
else:
    print("경고: 합칠 데이터가 없습니다. 파일 패턴을 확인하세요.")


모든 파일을 합쳐서 저장 완료: ../assets/xyz/merged_imp_gpu_label.xyzc


In [3]:
import glob
import numpy as np
import re
import os

# imp001~imp026 번호를 추출
def extract_layer(fpath):
    name = os.path.basename(fpath)
    m = re.match(r'imp0*([1-9]|1\d|2[0-6])_gpu\.xyz$', name)
    if not m:
        return None
    return int(m.group(1))

# 1) glob 으로 imp*_gpu.xyz 전부 뽑고
raw_list = glob.glob("../assets/xyz/imp*_gpu.xyz")

# 2) 번호가 1~26인 것만 필터링
file_list = []
for fp in raw_list:
    layer = extract_layer(fp)
    if layer is not None and 1 <= layer <= 26:
        file_list.append((layer, fp))

# 3) 번호 순으로 정렬
file_list.sort(key=lambda x: x[0])

all_data = []
for _, fp in file_list:
    data = np.loadtxt(fp)
    # 혹시 3열 이상이면 x,y,z만
    if data.ndim == 2 and data.shape[1] > 3:
        data = data[:, :3]
    elif data.ndim == 1:
        data = data[np.newaxis, :]
    all_data.append(data)

if all_data:
    merged = np.vstack(all_data)
    output_path = "../assets/xyz/merged_imp_gpu.xyz"
    fmt = ['%.18e', '%.18e', '%.18e']
    np.savetxt(output_path, merged, fmt=fmt)
    print(f"imp001~imp026 .xyz 파일을 합쳐서 저장 완료: {output_path}")
else:
    print("경고: imp001~imp026 .xyz 파일을 찾을 수 없습니다.")


imp001~imp026 .xyz 파일을 합쳐서 저장 완료: ../assets/xyz/merged_imp_gpu.xyz


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from ply.PointNet_seg import PointNetSeg  # 또는 PointNetImpellerSeg

model = PointNetSeg(num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1000):  # 충분히 과적합
    for xyz_file, label_file in dataset:
        points = torch.tensor(np.loadtxt(xyz_file)).float().to(device)  # (N, 3)
        labels = torch.tensor(np.load(label_file)).long().to(device)    # (N,)

        points = points.T.unsqueeze(0)  # (1, 3, N)
        labels = labels.unsqueeze(0)    # (1, N)

        pred = model(points)  # (1, C, N)
        loss = criterion(pred.squeeze(0), labels.squeeze(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
